In [1]:
from braket.emulation.device_emulator_properties import DeviceEmulatorProperties
# from braket.circuits.result_type import ResultType
from braket.device_schema.result_type import ResultType
from braket.emulation.local_emulator import LocalEmulator

from braket.device_schema.standardized_gate_model_qpu_device_properties_v1 import (
    CoherenceTime,
    Fidelity1Q,
    FidelityType,
    OneQubitProperties,
    TwoQubitProperties,
    GateFidelity2Q,
)

oneQubitProperties = OneQubitProperties(
    T1=CoherenceTime(value=2e-5, standardError=None, unit="S"),
    T2=CoherenceTime(value=8e-6, standardError=None, unit="S"),
    oneQubitFidelity=[
        Fidelity1Q(
            fidelityType=FidelityType(name="RANDOMIZED_BENCHMARKING", description=None),
            fidelity=0.99,
            standardError=None,
        ),
        Fidelity1Q(
            fidelityType=FidelityType(name="READOUT", description=None),
            fidelity=0.9795,
            standardError=None,
        ),
    ],
)

twoQubitProperties = TwoQubitProperties(
    twoQubitGateFidelity=[
        GateFidelity2Q(
            direction=None,
            gateName="CZ",
            fidelity=0.99,
            standardError=0.0009,
            fidelityType=FidelityType(name="RANDOMIZED_BENCHMARKING", description=None),
        ),
        GateFidelity2Q(
            direction=None,
            gateName="ISwap",
            fidelity=0.99,
            standardError=0.0009,
            fidelityType=FidelityType(name="RANDOMIZED_BENCHMARKING", description=None),
        ),
    ]
)

device_em_properties = DeviceEmulatorProperties(
    qubitCount = 2,
    nativeGateSet = ['rx', 'rz', 'iswap'],
    connectivityGraph = {},
    supportedResultTypes = [ResultType(name='Sample', observables=['x', 'y', 'z', 'h', 'i'], minShots=1, maxShots=5000)],
    oneQubitProperties = {"0": oneQubitProperties, "1": oneQubitProperties},
    twoQubitProperties = {"0-1": twoQubitProperties},
)


local_emu = LocalEmulator.from_device_properties(device_em_properties)

In [5]:
from braket.circuits.circuit import Circuit
from numpy import pi
circ = Circuit().rx(0,0).rz(1,pi).iswap(0,1)
circ = Circuit().add_verbatim_box(circ)
print(circ)

local_emu.run(circ, shots=10).result()

T  : │        0        │     1      │    2    │       3       │
                         ┌───────┐   ┌───────┐                 
q0 : ───StartVerbatim────┤ Rx(0) ├───┤ ISWAP ├───EndVerbatim───
              ║          └───────┘   └───┬───┘        ║        
              ║         ┌──────────┐ ┌───┴───┐        ║        
q1 : ─────────╨─────────┤ Rz(3.14) ├─┤ ISWAP ├────────╨────────
                        └──────────┘ └───────┘                 
T  : │        0        │     1      │    2    │       3       │


GateModelQuantumTaskResult(task_metadata=TaskMetadata(braketSchemaHeader=BraketSchemaHeader(name='braket.task_result.task_metadata', version='1'), id='8580eccf-8b4d-4cbc-a9e8-cc767c0b1044', shots=10, deviceId='braket_dm', deviceParameters=None, createdAt=None, endedAt=None, status=None, failureReason=None), additional_metadata=AdditionalMetadata(action=Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.openqasm.program', version='1'), source='OPENQASM 3.0;\nbit[2] b;\nqubit[2] q;\nrx(0.0) q[0];\n#pragma braket noise depolarizing(0.010000000000000009) q[0]\nrz(3.141592653589793) q[1];\n#pragma braket noise depolarizing(0.010000000000000009) q[1]\niswap q[0], q[1];\n#pragma braket noise two_qubit_depolarizing(0.010000000000000009) q[0], q[1]\n#pragma braket noise bit_flip(0.020499999999999963) q[0]\nb[0] = measure q[0];\n#pragma braket noise bit_flip(0.020499999999999963) q[1]\nb[1] = measure q[1];', inputs={}), dwaveMetadata=None, ionqMetadata=None, rigettiMetadata=None, oqcM